```{contents}
```
## Cost Optimization 

**Cost optimization in LangGraph** is the systematic design of agent workflows, execution graphs, and infrastructure to **minimize LLM, tool, memory, and compute costs** while preserving correctness, reliability, and performance.

In production systems, **90%+ of operational cost comes from model calls and token usage**, making cost optimization a **first-class architectural concern**.

---

### **1. Where Costs Originate in LangGraph**

| Cost Source        | Description                 |
| ------------------ | --------------------------- |
| LLM Inference      | Prompt + completion tokens  |
| Tool Calls         | External API usage          |
| Vector Search      | Embeddings + queries        |
| State Storage      | Checkpoints, memory, logs   |
| Execution Overhead | Retries, loops, concurrency |
| Infrastructure     | CPU, RAM, networking        |

---

### **2. Core Cost-Optimization Principles**

| Principle               | Purpose                 |
| ----------------------- | ----------------------- |
| Minimize Tokens         | Reduce LLM spend        |
| Reduce Model Calls      | Fewer invocations       |
| Use Cheaper Models      | Route intelligently     |
| Shorten Execution Paths | Less runtime            |
| Cache Aggressively      | Avoid recomputation     |
| Terminate Early         | Avoid unnecessary loops |

---

### **3. Token-Level Optimization**

#### **Prompt Compression**

```python
def compact_prompt(state):
    return {
        "messages": state["messages"][-3:]  # keep only last 3 turns
    }
```

#### **Structured Output**

```python
system_prompt = "Respond in strict JSON. No explanations."
```

Reduces verbosity → **30–50% token savings**.

---

### **4. Model Routing Strategy**

```python
def choose_model(state):
    if state["complexity"] < 0.3:
        return "gpt-3.5"
    return "gpt-4"
```

| Task Type            | Model         |
| -------------------- | ------------- |
| Simple extraction    | Cheap model   |
| Reasoning / planning | Premium model |
| Verification         | Cheap model   |

**Savings:** up to **70%** on inference cost.

---

### **5. Graph-Level Optimization**

#### **Early Exit Conditions**

```python
def router(state):
    if state["confidence"] > 0.9:
        return END
    return "refine"
```

#### **Cycle Budgeting**

```python
graph.invoke(input, config={"recursion_limit": 5})
```

Prevents runaway loops.

---

### **6. Caching & Memoization**

```python
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()
```

| Cache Layer     | Savings                   |
| --------------- | ------------------------- |
| Prompt cache    | Skip repeated calls       |
| Tool cache      | Avoid duplicate API calls |
| Embedding cache | Eliminate recomputation   |

---

### **7. Parallelization & Batching**

```python
builder.add_node("parallel_eval", async_node)
```

Batch LLM requests:

| Benefit       | Impact                |
| ------------- | --------------------- |
| Lower latency | Faster response       |
| Lower cost    | Fewer overhead tokens |

---

### **8. Checkpointing & Recovery**

Avoid re-running expensive steps:

```python
graph = builder.compile(checkpointer=PostgresSaver())
```

If crash → resume from checkpoint.

---

### **9. Cost Observability**

| Tool             | Role                  |
| ---------------- | --------------------- |
| Token meters     | Measure usage         |
| Per-run budgets  | Hard caps             |
| Alert thresholds | Prevent overruns      |
| Usage dashboards | Optimization feedback |

```python
graph.invoke(input, config={"max_cost": 0.02})
```

---

### **10. Enterprise Cost Control Architecture**

| Layer        | Optimization           |
| ------------ | ---------------------- |
| Prompt layer | Compression, structure |
| Graph layer  | Short paths, pruning   |
| Agent layer  | Smart routing          |
| Model layer  | Tiered models          |
| Memory layer | TTL, eviction          |
| Ops layer    | Autoscaling            |

---

### **11. Measurable Impact (Real Systems)**

| Optimization        | Cost Reduction |
| ------------------- | -------------- |
| Prompt compression  | 20–40%         |
| Model routing       | 40–70%         |
| Caching             | 50–90%         |
| Loop control        | 30–60%         |
| Checkpoint recovery | 20–50%         |

---

### **12. Mental Model**

> **Every unnecessary token is money.
> Every unnecessary node is cost.
> Every unnecessary loop is burn.**

Cost optimization in LangGraph is therefore **graph design + agent policy + runtime governance**.


### Demonstration

In [2]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_classic.cache import InMemoryCache
import langchain

# --------- Global Cache ---------
langchain.llm_cache = InMemoryCache()

# --------- State ---------
class State(TypedDict):
    query: str
    complexity: float
    answer: str
    confidence: float

# --------- Models ---------
cheap_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
strong_model = ChatOpenAI(model="gpt-4", temperature=0)

# --------- Cost-Aware Router ---------
def route_model(state: State):
    if state["complexity"] < 0.4:
        return "cheap"
    return "strong"

# --------- Nodes ---------
def solve_with_cheap(state):
    msg = f"Answer briefly: {state['query']}"
    resp = cheap_model.invoke(msg)
    return {"answer": resp.content, "confidence": 0.7}

def solve_with_strong(state):
    msg = f"Answer concisely with JSON: {{'answer': ...}}. Question: {state['query']}"
    resp = strong_model.invoke(msg)
    return {"answer": resp.content, "confidence": 0.95}

def should_stop(state):
    if state["confidence"] > 0.9:
        return END
    return "strong"

# --------- Graph ---------
builder = StateGraph(State)

builder.add_node("cheap", solve_with_cheap)
builder.add_node("strong", solve_with_strong)

builder.set_entry_point("cheap")

builder.add_conditional_edges("cheap", route_model, {
    "cheap": END,
    "strong": "strong"
})

builder.add_conditional_edges("strong", should_stop, {
    "strong": "strong",
    END: END
})

graph = builder.compile()

# --------- Run with Loop & Cost Control ---------
result = graph.invoke(
    {"query": "Explain black holes simply", "complexity": 0.6},
    config={"recursion_limit": 3}
)

print(result)


{'query': 'Explain black holes simply', 'complexity': 0.6, 'answer': '{\'answer\': "Black holes are regions in space where gravity is so strong that nothing, not even light, can escape from them. They are formed when a large star collapses under its own gravity after its life cycle ends. Inside a black hole, all the matter is squeezed into a tiny space, creating a point of infinite density called a singularity."}', 'confidence': 0.95}
